# Bike_sharing

# Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(y)
Y[:5]

# 2, Regression- np.sqrt(mse)


In [1]:
import pandas as pd
DF = pd.read_csv('https://raw.githubusercontent.com/rusita-ai/pyData/master/Bike_Sharing_Demand.csv')
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [2]:
DF.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
X = DF[['season','holiday','workingday','weather','temp','humidity','windspeed']]
y = DF['count']

# scaling

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
DF[['season','holiday','workingday','weather','temp','humidity','windspeed']] = scaler.fit_transform(X)

## 1, Multivariate Regression

### 모델링

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

In [7]:
def multiReg(model, X, y, dg):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  MR = LinearRegression(normalize=True, n_jobs=-1)
  MR.fit(PX_tr, y_train)

  # evaluate
  MR_mse = np.sqrt(mean_squared_error(y_test, MR.predict(PX_te)))

  return {'model':model, 'degree':dg, 'mse':MR_mse}

### 모델평가(degree)

In [8]:
%%time
MR_mse = pd.DataFrame(columns=['model','degree','mse'])
a =[1,2,5,7]
index= 1

for i in a:
  model = 'MR-'+str(index)
  index += 1
  res = multiReg(model, X, y, dg=i)
  print(res)
  MR_mse = MR_mse.append(res, ignore_index=True)


{'model': 'MR-1', 'degree': 1, 'mse': 156.03395296591316}
{'model': 'MR-2', 'degree': 2, 'mse': 151.93139856165664}
{'model': 'MR-3', 'degree': 5, 'mse': 1295827594544.5159}
{'model': 'MR-4', 'degree': 7, 'mse': 363427590107.84955}
CPU times: user 1min 3s, sys: 2.3 s, total: 1min 5s
Wall time: 33.8 s


## 2, Ridge Regression

### 모델링

In [12]:
from sklearn.linear_model import Ridge

def ridge(model, X, y, dg, ap=0.3):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  RG = Ridge(normalize=True, alpha=ap, solver='cholesky')
  RG.fit(PX_tr, y_train)

  RG_mse = np.sqrt(mean_squared_error(y_test, RG.predict(PX_te)))

  return {'model':model, 'degree':dg, 'alpha':ap, 'mse':RG_mse}

### 모델평가(degree)

In [13]:
%%time
RG_mse = pd.DataFrame(columns=['model','degree','alpha','mse'])
a = [1,2,3,4]
index = 1

for i in a:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)

{'model': 'RG-1', 'degree': 1, 'alpha': 0.3, 'mse': 157.08589772713447}
{'model': 'RG-2', 'degree': 2, 'alpha': 0.3, 'mse': 155.31154849885817}
{'model': 'RG-3', 'degree': 3, 'alpha': 0.3, 'mse': 154.28456797646976}
{'model': 'RG-4', 'degree': 4, 'alpha': 0.3, 'mse': 153.33822329027635}


In [14]:
%%time
for i in [5,6,7]:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)

{'model': 'RG-5', 'degree': 5, 'alpha': 0.3, 'mse': 152.42341799563565}
{'model': 'RG-6', 'degree': 6, 'alpha': 0.3, 'mse': 151.68938013839383}
{'model': 'RG-7', 'degree': 7, 'alpha': 0.3, 'mse': 151.1928425035353}


In [15]:
%%time
for i in [8,9,10]:
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)

{'model': 'RG-8', 'degree': 8, 'alpha': 0.3, 'mse': 150.91523216210038}
{'model': 'RG-9', 'degree': 9, 'alpha': 0.3, 'mse': 150.85883200355232}
{'model': 'RG-10', 'degree': 10, 'alpha': 0.3, 'mse': 151.7777399153868}


In [17]:
RG_mse.sort_values(by='mse')

,model,degree,alpha,mse
8,RG-9,9,0.3,150.858832
7,RG-8,8,0.3,150.915232
6,RG-7,7,0.3,151.192843
5,RG-6,6,0.3,151.689380
9,RG-10,10,0.3,151.777740
4,RG-5,5,0.3,152.423418
3,RG-4,4,0.3,153.338223
2,RG-3,3,0.3,154.284568
1,RG-2,2,0.3,155.311548
0,RG-1,1,0.3,157.085898


### 모델평가(alpha)

In [18]:
%%time
a = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9]

for i in a :
  model = 'RG-'+str(index)
  index += 1
  res = ridge(model, X, y, dg=9 , ap=i)
  print(res)
  RG_mse = RG_mse.append(res, ignore_index=True)

{'model': 'RG-11', 'degree': 9, 'alpha': 0.01, 'mse': 179.7782994897138}
{'model': 'RG-12', 'degree': 9, 'alpha': 0.05, 'mse': 154.93449616144116}
{'model': 'RG-13', 'degree': 9, 'alpha': 0.1, 'mse': 151.53889800742218}
{'model': 'RG-14', 'degree': 9, 'alpha': 0.3, 'mse': 150.85883200355232}
{'model': 'RG-15', 'degree': 9, 'alpha': 0.5, 'mse': 151.29084786348537}
{'model': 'RG-16', 'degree': 9, 'alpha': 0.7, 'mse': 151.70487589594217}
{'model': 'RG-17', 'degree': 9, 'alpha': 0.9, 'mse': 152.07164146811968}


In [22]:
RG_mse.sort_values(by='mse').head(3)

,model,degree,alpha,mse
8,RG-9,9,0.3,150.858832
13,RG-14,9,0.3,150.858832
7,RG-8,8,0.3,150.915232


## 3, Lasso

### 모델링

In [23]:
from sklearn.linear_model import Lasso

def lasso(model, X, y, dg, ap=0.3):
  # spint
  X_train, X_test, y_train ,y_test = train_test_split(X, y, test_size=0.3, random_state=2045)

  # poly
  poly = PolynomialFeatures(degree=dg, include_bias=False)
  PX_tr = poly.fit_transform(X_train)
  PX_te = poly.fit_transform(X_test)

  # modeling
  LS = Lasso(normalize=True, alpha=ap)
  LS.fit(PX_tr, y_train)

  # mse
  LS_mse = np.sqrt(mean_squared_error(y_test, LS.predict(PX_te)))  

  return {'model':model, 'degree':dg, 'alpha':ap, 'mse':LS_mse}

### 모델평가(degree)

In [26]:
%%time
LS_mse = pd.DataFrame(columns=['model','degree','alpha','mse'])
a = [1,2,3,4,5,6,7,8,9,10]
b = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9]
index = 1

for i in a :
  for j in b:
    model = 'LS-'+str(index)
    index += 1
    res = lasso(model, X, y, dg=i, ap=j)
    print(res)
    LS_mse = LS_mse.append(res, ignore_index=True)

{'model': 'LS-1', 'degree': 1, 'alpha': 0.01, 'mse': 156.03483114148662}
{'model': 'LS-2', 'degree': 1, 'alpha': 0.1, 'mse': 156.67968077801606}
{'model': 'LS-3', 'degree': 1, 'alpha': 0.3, 'mse': 161.47174247042977}
{'model': 'LS-4', 'degree': 1, 'alpha': 0.5, 'mse': 168.03552338148305}
{'model': 'LS-5', 'degree': 1, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-6', 'degree': 1, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-7', 'degree': 2, 'alpha': 0.01, 'mse': 152.7828900990057}
{'model': 'LS-8', 'degree': 2, 'alpha': 0.1, 'mse': 156.21371823141956}
{'model': 'LS-9', 'degree': 2, 'alpha': 0.3, 'mse': 160.77439051265412}
{'model': 'LS-10', 'degree': 2, 'alpha': 0.5, 'mse': 167.8895773520336}
{'model': 'LS-11', 'degree': 2, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-12', 'degree': 2, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-13', 'degree': 3, 'alpha': 0.01, 'mse': 151.8050696992396}
{'model': 'LS-14', 'degree': 3, 'alpha': 0.1, 'mse': 155.665

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 55053.87012270093, tolerance: 25086.59172675853
  positive)


{'model': 'LS-25', 'degree': 5, 'alpha': 0.01, 'mse': 151.02530671713856}
{'model': 'LS-26', 'degree': 5, 'alpha': 0.1, 'mse': 155.07752679080843}
{'model': 'LS-27', 'degree': 5, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-28', 'degree': 5, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-29', 'degree': 5, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-30', 'degree': 5, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-31', 'degree': 6, 'alpha': 0.01, 'mse': 150.73470858590477}
{'model': 'LS-32', 'degree': 6, 'alpha': 0.1, 'mse': 154.75922772931125}
{'model': 'LS-33', 'degree': 6, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-34', 'degree': 6, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-35', 'degree': 6, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-36', 'degree': 6, 'alpha': 0.9, 'mse': 180.42555843739754}
{'model': 'LS-37', 'degree': 7, 'alpha': 0.01, 'mse': 150.50192402700253}
{'model': 'LS-38', 'degree': 7, 'alpha': 0.1, 'm

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 102433.59030893445, tolerance: 25086.59172675853
  positive)


{'model': 'LS-55', 'degree': 10, 'alpha': 0.01, 'mse': 150.72288527814004}
{'model': 'LS-56', 'degree': 10, 'alpha': 0.1, 'mse': 154.38258928925845}
{'model': 'LS-57', 'degree': 10, 'alpha': 0.3, 'mse': 160.64199870421058}
{'model': 'LS-58', 'degree': 10, 'alpha': 0.5, 'mse': 167.85306675897215}
{'model': 'LS-59', 'degree': 10, 'alpha': 0.7, 'mse': 176.51473428440806}
{'model': 'LS-60', 'degree': 10, 'alpha': 0.9, 'mse': 180.42555843739754}


In [34]:
LS_mse.sort_values(by='mse').head(3)

,model,degree,alpha,mse
42,LS-43,8,0.01,150.441037
36,LS-37,7,0.01,150.501924
48,LS-49,9,0.01,150.567032


## 4, Decision Tree

### 모델링

In [32]:
from sklearn.tree import DecisionTreeRegressor

def decisionTree(model, X, y, md):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2045)

  # modeling
  DT = DecisionTreeRegressor(max_depth=md, criterion='mse')
  DT.fit(X_train, y_train)

  DT_mse = np.sqrt(mean_squared_error(y_test, DT.predict(X_test)))

  return {'model':model, 'max_depth':md, 'mse':DT_mse}

### 모델링(max_depth)

In [33]:
DT_mse = pd.DataFrame(columns=['model','max_depth','mse'])
a = [2,3,5,7,9]
index = 1

for i in a:
  model = 'DT-'+str(index)
  index += 1
  res = decisionTree(model, X, y, md=i)
  print(res)
  DT_mse = DT_mse.append(res, ignore_index=True)

{'model': 'DT-1', 'max_depth': 2, 'mse': 160.74168249273998}
{'model': 'DT-2', 'max_depth': 3, 'mse': 156.0859197046373}
{'model': 'DT-3', 'max_depth': 5, 'mse': 151.6448906761661}
{'model': 'DT-4', 'max_depth': 7, 'mse': 149.83337137828704}
{'model': 'DT-5', 'max_depth': 9, 'mse': 154.74630523430213}


## 5, Random Forest

### 모델링

In [46]:
from sklearn.ensemble import RandomForestRegressor

def randomForest(model, X, y, n_est, md='auto', mf=None, visualFeatures=False):
  # split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2045)

  # modeling
  RF = RandomForestRegressor(n_estimators=n_est, max_depth=md, max_features=mf, criterion='mse', n_jobs=-1, random_state=2045)
  RF.fit(X_train, y_train)

  # mse
  RF_mse = np.sqrt(mean_squared_error(y_test, RF.predict(X_test)))

  if visualFeatures:
    sns.barplot(RF.feature_importances_, X.columns)
    plt.title('random forest feature importances')
    plt.show()

  return {'model':model, 'n_estimators':n_est, 'max_depth':md, 'max_features':mf, 'mse':RF_mse}

In [47]:
RF_mse = pd.DataFrame(columns=['model','n_estimators','max_depth','max_features','mse'])
a = [50, 100, 200, 300, 500]
index = 1

for i in a:
  model = 'RF-'+str(index)
  index += 1
  res = randomForest(model, X, y, n_est=i)
  print(res)
  RF_mse = RF_mse.append(res, ignore_index=True)

TypeError: ignored